In [ ]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from tensorflow.python.ops.distributions.categorical import Categorical
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

### 데이터 읽어오기

In [ ]:
def extract(df):
    X = df.drop(columns=['MEDV'])
    Y = df['MEDV']
    return X, Y


def readCSV(fileName):
    df = (pd.read_csv(fileName,
                      dtype={'CHAS': 'bool',
                             'RAD': 'category',
                             'TAX': 'int'})
          .set_index('ID'))
    
    radType = CategoricalDtype(categories=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24'], ordered=True)
    df['RAD'] = df['RAD'].astype(radType)
    return df


# 데이터 읽기
data = readCSV('data/train.csv')
test = readCSV('data/test.csv')
input_x, input_y = extract(data)
submit_x = test

### 검증

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# 검증
model = RandomForestRegressor(random_state=42)
rmse_scores = -cross_val_score(model, input_x, input_y, scoring='neg_root_mean_squared_error', cv=5)

# 평균 RMSE 출력
print(f"Average RMSE: {np.mean(rmse_scores):.4f}")

### 훈련 및 예측 파일 생성

In [ ]:
# 훈련
model = RandomForestRegressor(random_state=42)
model.fit(input_x, input_y)

# 결과 파일 생성
submit_y = pd.DataFrame(model.predict(submit_x), index=submit_x.index, columns=['MEDV'])
submit_y.to_csv("submission.csv", index=True)